# 1.Vector Store Retriever

In [5]:
from langchain_community.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.documents import Document

In [6]:
# Step 1: Your source code docs
documents = [
    Document(page_content="LangChain helps developers build LLM applications easily."),
    Document(page_content="Chroma is a vector database optimized for LLM-based search."),
    Document(page_content="Embeddings convert text into high-dimensional vectors."),
    Document(page_content="OpenAI provides powerful embedding models."),
]

In [7]:
# Initialize embedding model
embedding_model = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")


# Step 3: Create Chroma vector store in memory
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    collection_name="my_collection"
)

In [9]:
# Convert vectorstore into a retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

In [ ]:
query= "words converted into embeddings are called?"
results = retriever.invoke(query)  # runnable object can be run in chain

In [13]:
for i,doc in enumerate(results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
Embeddings convert text into high-dimensional vectors.

--- Result 2 ---
OpenAI provides powerful embedding models.


In [14]:
# or 
results = vectorstore.similarity_search(query,k=2)

In [15]:
for i,doc in enumerate(results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
Embeddings convert text into high-dimensional vectors.

--- Result 2 ---
OpenAI provides powerful embedding models.


# 2.MMR

In [27]:
# sample docuements
docs = [
    Document(page_content="LangChain makes it easy to work with LLMs."),
    Document(page_content="LangChain is used to build LLM based applications."),
    Document(page_content="Chroma is used to store and search document embeddings."),
    Document(page_content="Embeddings are vector representations of text."),
    Document(page_content="MMR helps you get diverse results when doing similarity search."),
    Document(page_content="LangChain supports Chroma, FAISS, Pinecone, and more."),
]

In [ ]:
from langchain_community.vectorstores import FAISS   # vector store from Meta (facebook)

# Initialize Gemini embeddings
embedding_model = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

# Create the FAISS vector_store from documents
vectorstore = FAISS.from_documents(
    documents=docs,
    embedding=embedding_model
)

In [29]:
# Enable MMR in the Retriever
retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={"k":3, "lambda_mult":0.5} # k = top results, lambda_mult = relevance-diversity balance
)

In [30]:
query = "What is Langchain?"
results = retriever.invoke(query)

In [ ]:
for i, doc in enumerate(results):
    print(f"\n --- Result {i+1} ---")
    print(doc.page_content)


 --- Result 1 ---
LangChain is used to build LLM based applications.

 --- Result 2 ---
MMR helps you get diverse results when doing similarity search.

 --- Result 3 ---
LangChain supports Chroma, FAISS, Pinecone, and more.


# 2.2 MMR - Maximum Marginal Relevance
#### - Using HuggingFace Embeddings!

In [ ]:
from langchain_community.vectorstores import FAISS

from langchain_huggingface import HuggingFaceEmbeddings, HuggingFaceEndpointEmbeddings
# from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint

# from langchain_google_genai import GoogleGenerativeAIEmbeddings
# from langchain_openai import OpenAIEmbeddings
# from langchain_google_genai import ChatGoogleGenerativeAI

from langchain.retrievers.multi_query import MultiQueryRetriever
from dotenv import load_dotenv
load_dotenv()

True

In [37]:
# sample docuements
docs = [
    Document(page_content="LangChain makes it easy to work with LLMs."),
    Document(page_content="LangChain is used to build LLM based applications."),
    Document(page_content="Chroma is used to store and search document embeddings."),
    Document(page_content="Embeddings are vector representations of text."),
    Document(page_content="MMR helps you get diverse results when doing similarity search."),
    Document(page_content="LangChain supports Chroma, FAISS, Pinecone, and more."),
]

In [38]:
# llm=HuggingFaceEndpoint(
#     repo_id="google/gemma-2-2b-it",     #For MQR if required a llm and if wanted to used gemma.
#     task="text-generation"
# )
# model1 = ChatHuggingFace(llm=llm)

embeddings = HuggingFaceEndpointEmbeddings(repo_id="sentence-transformers/all-MiniLM-L6-v2")

# Create the FAISS vector_store from documents
vectorstore = FAISS.from_documents(
    documents=docs,
    embedding=embedding_model
)

In [39]:
retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={"k":3, "lambda_mult":0.5}
)

In [40]:
query = "What is Langchain?"
results = retriever.invoke(query)

In [42]:
for i, doc in enumerate(results):
    print(f"\n --- Result {i+1} ---")
    print(doc.page_content)


 --- Result 1 ---
LangChain is used to build LLM based applications.

 --- Result 2 ---
MMR helps you get diverse results when doing similarity search.

 --- Result 3 ---
LangChain supports Chroma, FAISS, Pinecone, and more.


# 3.Multi Query Retriever 
#### - using HF (sentence-transformers/all-MiniLM-L6-v2) model

In [43]:
from langchain_community.vectorstores import FAISS

from langchain_huggingface import HuggingFaceEmbeddings, HuggingFaceEndpointEmbeddings
# from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint

from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
# from langchain_openai import OpenAIEmbeddings

from langchain.retrievers.multi_query import MultiQueryRetriever
from dotenv import load_dotenv
load_dotenv()

True

In [44]:
# Relevant health & wellness documents
all_docs = [
    Document(page_content="Regular walking boosts heart health and can reduce symptoms of depression.", metadata={"source": "H1"}),
    Document(page_content="Consuming leafy greens and fruits helps detox the body and improve longevity.", metadata={"source": "H2"}),
    Document(page_content="Deep sleep is crucial for cellular repair and emotional regulation.", metadata={"source": "H3"}),
    Document(page_content="Mindfulness and controlled breathing lower cortisol and improve mental clarity.", metadata={"source": "H4"}),
    Document(page_content="Drinking sufficient water throughout the day helps maintain metabolism and energy.", metadata={"source": "H5"}),
    Document(page_content="The solar energy system in modern homes helps balance electricity demand.", metadata={"source": "I1"}),
    Document(page_content="Python balances readability with power, making it a popular system design language.", metadata={"source": "I2"}),
    Document(page_content="Photosynthesis enables plants to produce energy by converting sunlight.", metadata={"source": "I3"}),
    Document(page_content="The 2022 FIFA World Cup was held in Qatar and drew global energy and excitement.", metadata={"source": "I4"}),
    Document(page_content="Black holes bend spacetime and store immense gravitational energy.", metadata={"source": "I5"}),
]

In [45]:
# Hugging Face Embeddings
embeddings = HuggingFaceEndpointEmbeddings(repo_id="sentence-transformers/all-MiniLM-L6-v2")

# Create FAISS vector store
vectorstore = FAISS.from_documents(documents=all_docs, embedding=embeddings)

In [ ]:
# Create retrievers (Normal! )
similarity_retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":5})

In [53]:
# Multiquery Retriever 
# along wiht llm to ask more questions to get more info about query to an llm before find it's vector using retriever
mulitquery_retriever = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(search_kwargs={"k":5}), llm=ChatGoogleGenerativeAI(model='gemini-2.0-flash')
)

In [54]:
# Query
query = "HOw to improve energy levels and maintain balance"

In [55]:
# Retriever Results
similarity_results = similarity_retriever.invoke(query)
mulitquery_results = mulitquery_retriever.invoke(query)

In [57]:
for i, doc in enumerate(similarity_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)

print("*"*150)

for i, doc in enumerate(mulitquery_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)



--- Result 1 ---
Drinking sufficient water throughout the day helps maintain metabolism and energy.

--- Result 2 ---
The solar energy system in modern homes helps balance electricity demand.

--- Result 3 ---
Consuming leafy greens and fruits helps detox the body and improve longevity.

--- Result 4 ---
Mindfulness and controlled breathing lower cortisol and improve mental clarity.

--- Result 5 ---
Photosynthesis enables plants to produce energy by converting sunlight.
******************************************************************************************************************************************************

--- Result 1 ---
The solar energy system in modern homes helps balance electricity demand.

--- Result 2 ---
Drinking sufficient water throughout the day helps maintain metabolism and energy.

--- Result 3 ---
Photosynthesis enables plants to produce energy by converting sunlight.

--- Result 4 ---
Python balances readability with power, making it a popular system desi

# Contextual Compression Retriever
- using Hf embeddings 

In [67]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFaceEndpointEmbeddings

from langchain_google_genai import ChatGoogleGenerativeAI
# from langchain_openai import OpenAIEmbeddings

from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

from dotenv import load_dotenv
load_dotenv()

True

In [68]:
# Recreate the document objects from the previous data
docs = [
    Document(page_content=(
        """The Grand Canyon is one of the most visited natural wonders in the world.
        Photosynthesis is the process by which green plants convert sunlight into energy.
        Millions of tourists travel to see it every year. The rocks date back millions of years."""
    ), metadata={"source": "Doc1"}),

    Document(page_content=(
        """In medieval Europe, castles were built primarily for defense.
        The chlorophyll in plant cells captures sunlight during photosynthesis.
        Knights wore armor made of metal. Siege weapons were often used to breach castle walls."""
    ), metadata={"source": "Doc2"}),

    Document(page_content=(
        """Basketball was invented by Dr. James Naismith in the late 19th century.
        It was originally played with a soccer ball and peach baskets. NBA is now a global league."""
    ), metadata={"source": "Doc3"}),

    Document(page_content=(
        """The history of cinema began in the late 1800s. Silent films were the earliest form.
        Thomas Edison was among the pioneers. Photosynthesis does not occur in animal cells.
        Modern filmmaking involves complex CGI and sound design."""
    ), metadata={"source": "Doc4"})
]

In [69]:
# Create a FAISS vector store from the documents
# Hugging Face Embeddings
embeddings = HuggingFaceEndpointEmbeddings(repo_id="sentence-transformers/all-MiniLM-L6-v2")

# Create FAISS vector store
vectorstore = FAISS.from_documents(documents=docs, embedding=embeddings)

In [70]:
base_retriever = vectorstore.as_retriever(search_kwargs={"k":5})

In [71]:
# Setting up the Compressor using LLM
llm= ChatGoogleGenerativeAI(model='gemini-1.5-flash')
compressor = LLMChainExtractor.from_llm(llm=llm)

In [73]:
# Creating Contexual Compression (Retriever)
compression_retriever=ContextualCompressionRetriever(
    base_retriever= base_retriever,
    base_compressor=compressor
)

In [74]:
# Query the retriever
query = "what is photosynthesis?"
compressed_results = compression_retriever.invoke(query)

In [75]:
for i, doc in enumerate(compressed_results):
    print(f"\n--- Results {i+1} ---")
    print(doc.page_content)


--- Results 1 ---
Photosynthesis is the process by which green plants convert sunlight into energy.

--- Results 2 ---
The chlorophyll in plant cells captures sunlight during photosynthesis.

--- Results 3 ---
Photosynthesis does not occur in animal cells.
